# Preparation of Weather Data
The following notebook demonstrates how weather data was scraped from airdensity online. A site that provides free to use weather data for corresponding race tracks. In this instance, Formula 1 circuits.

The site can be found [here.]('https://airdensityonline.com/tracks/')

## Imports & Helper Methods

### Import Modules

In [15]:
import requests
import csv
from bs4 import BeautifulSoup as bs
import os

### 12 Hour to 24 Hour Time Fuction
Converts the scraped time format into more readble 24-hour time.

In [16]:
# Function to convert 12-hour time format to 24-hour
def convert24(raw): 
      
    # Checking if last two elements of time 
    # is AM and first two elements are 12 
    if raw[-2:] == "am" and raw[:2] == "12": 
        return "00" + raw[2:-3] + ":00"
    
    # Check for e.g. 10:00 am, 11:00 am
    elif raw[-2:] == "am" and (raw[:2] == "11" or raw[:2] == "10"):
        return raw[:-3] + ":00"
        
    # remove the AM     
    elif raw[-2:] == "am": 
        return "0" + raw[:-3] + ":00"
      
    # Checking if last two elements of time 
    # is PM and first two elements are 12    
    elif raw[-2:] == "pm" and raw[:2] == "12": 
        return raw[:5] + ":00"
          
    elif raw[2] == ":": 
        # add 12 to hours and remove PM 
        return str(int(raw[:2]) + 12) + raw[2:5] + ":00"
    
    else: 
        # add 12 to hours and remove PM 
        return str(int(raw[:1]) + 12) + ":" + raw[2:4] + ":00"

### Fahrenheit to Celsius Function
Changes temp from F to C.

In [17]:
def f_to_c(temp):
    
    if temp != "inf":
        return round((int(temp) - 32) * 5/9)
    else:
        return 'null'

f_to_c("inf")

'null'

### Check for negative or nan value

Checks specifically the grain for negative or nan value.

In [18]:
def check_neg_nan(string):
    
    if string == "nan":
        return 'null'
    elif float(string) < 0:
        return 'null'
    else:
        return string
    
check_neg_nan("nan")

'null'

## Scrape Website Weather Data
Here we scrape the data from the weather website, modifying certain attributes for easier processing.
You are able to specify the url of the site, and path you want to save the file for.

**Note: This only works for links such as https://airdensityonline.com/track-history/Melbourne_Grand_Prix_Circuit/2020-03-21/**

In [21]:
def scrape_weather(url, path):
    # Prepare response raw data
    response = requests.get(url)
    soup = bs(response.content, 'html.parser')
    rows = soup.find(class_ = 'forecastdata').find_all('ul', recursive=False)[1:]

    # Create empty csv for writing
    file = open('data/' + path, 'w', newline='')
    writer = csv.writer(file)
    writer.writerow(['time', 'temp', 'dew_point', 'humidity', 'grains', 'wind_speed', 'wind_direction'])

    # Iterate over rows and scrape data
    for row in rows:
        time = row.find_all('li')[0].text.strip()[7:]
        temp = row.find_all('li')[1].text.strip()[:-6]
        dew_point = row.find_all('li')[4].text.strip()[:-6]
        humidity = row.find_all('li')[2].text.strip()[:-1]
        grains = row.find_all('li')[5].text.strip()
        wind = row.find_all('li')[6].text.strip()[:-4]
        wind_speed = wind.split()[1]
        wind_direction = wind.split()[0]

        # Convert particular data
        time = convert24(time)
        temp = f_to_c(temp)
        dew_point = f_to_c(dew_point)
        grains = check_neg_nan(grains)
        
        writer.writerow([time, temp, dew_point, humidity, grains, wind_speed, wind_direction])
    
    file.close()
    return "File Generated. Size: " + str(os.stat('data/' + path).st_size)

## 

In [22]:
scrape_weather('https://airdensityonline.com/track-history/Autodromo_de_Interlagos_Portao/2018-11-11/', 'weather/test.csv')

'File Generated. Size: 766'

In [ ]:

british_gp_url = ['2016', '2017', '2018', '2019']
british_gp_dates = "https://airdensityonline.com/track-history/Autodromo_de_Interlagos_Portao/"
